In [2]:
import pandas as pd
import numpy as np
import sklearn

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, balanced_accuracy_score

import pymorphy2

from joblib import dump, load
import pickle

In [6]:
df_train = pd.read_csv('/Users/nbold/Desktop/sber/data/df_first.csv', index_col=0)

In [8]:
STOP_WORDS = ['ипотека', 'ипотеку', 'ипотеке', 'ипотеки', 'в', 'на', 'по', 'под', 'при', 'с']
MORPH = pymorphy2.MorphAnalyzer()
def get_lem(text):
    lemmas = [
        MORPH.parse(it)[0].normal_form
        for it in text.lower().split()
        if it not in STOP_WORDS
    ]
    return ' '.join(lemmas)

In [8]:
df_train['request'] = df_train['request'].apply(lambda value: get_lem(value))

In [9]:
X_train_text = df_train['request'].values
y_train = df_train['label'].values

In [10]:
v = TfidfVectorizer(norm=None, max_df=0.8, max_features=500, decode_error='replace')
X_train_vector = v.fit_transform(X_train_text)

In [18]:
with open("tfidf.model", 'wb') as fout:
    pickle.dump(v, fout)

In [11]:
clf = LogisticRegression(random_state=43, solver='lbfgs',
                         max_iter=10000, n_jobs=-1, class_weight='balanced')
clf.fit(X_train_vector, y_train);

LogisticRegression(class_weight='balanced', max_iter=10000, n_jobs=-1,
                   random_state=64)

In [15]:
dump(clf, 'sber.joblib');